In [ ]:
!pip install chromadb

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.3/67.3 kB 3.7 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.4/21.4 MB 38.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 278.2/278.2 kB 13.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 33.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 103.3/103.3 kB 5.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.4/17.4 MB 53.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.5/72.5 kB 5.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 132.4/132.4 kB 8.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 66.4/66.4 kB 5.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 220.0/220.0 kB 16.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 105.4/105.4 kB 7.8 MB/s eta 

In [ ]:
import os
import uuid
import time
import io
from typing import List
from fastapi import FastAPI, File, UploadFile, Form, HTTPException
from pydantic import BaseModel
from PIL import Image
import chromadb
from chromadb.config import Settings
from sentence_transformers import SentenceTransformer
from transformers import BlipProcessor, BlipForConditionalGeneration

print("All modules and libraries have been imported")

All modules and libraries have been imported


In [ ]:
from google.colab import drive

# Mount Google Drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
# load image from the IAM dataset
image_path = "/content/drive/MyDrive/Datasets/download.jpeg"
image = Image.open(image_path).convert("RGB")

In [ ]:
!pip install easyocr

  Using cached ninja-1.13.0-py3-none-manylinux2014_x86_64.manylinux_2_17_x86_64.whl.metadata (5.1 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.9/2.9 MB 29.9 MB/s eta 0:00:00
Using cached ninja-1.13.0-py3-none-manylinux2014_x86_64.manylinux_2_17_x86_64.whl (180 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 978.2/978.2 kB 52.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 300.6/300.6 kB 21.0 MB/s eta 0:00:00


In [ ]:
import easyocr

# Create an OCR reader object
reader = easyocr.Reader(['en'])

# Read text from an image
result = reader.readtext(image_path)

# Print the extracted text
for detection in result:
    print(detection[1])

Progress: |██████████████████████████████████████████████████| 100.0% Complete

Progress: |██████████████████████████████████████████████████| 100.0% Complete

/usr/local/lib/python3.12/dist-packages/torch/utils/data/dataloader.py:668: UserWarning: 'pin_memory' argument is set as true but no accelerator is found, then device pinned memory won't be used.
  warnings.warn(warn_msg)


Best.
Summer
Ever.


In [ ]:
!pip install groq

In [ ]:
from groq import Groq
import json

# create a tag extractor class.
class TagExtractor():

    # Load the GROQ API KEY from the env file.
    GROQ_API_KEY = "insert your groq api key here"

    client = Groq(api_key=GROQ_API_KEY)

    query:str
    # output:str = ""

    models = [
    # "llama-3.1-405b-reasoning",
    # "llama-3.1-70b-versatile",
    "llama-3.1-8b-instant",
    # "mixtral-8x7b-32768"
    ]

    token_class = {"short": 150, "moderate": 700, "long": 1536}

    system_prompt = """
                You are an intelligent generative assistant specialized in extracting relevant tags from multimedia content.
                Your goal is to provide concise, accurate, and descriptive tags that effectively capture the essence of the content.
                These tags will be used for indexing, search, and categorization.

                Rules:
                - Generate **3 to 4 tags** per content.
                - Tags must be concise, highly relevant, and effectively describe the content.
                - Return tags as a **JSON list of strings**.

                Few-shot examples:

                Example 1:
                Content: A high-tempo Afrobeats song designed for workouts.
                Tags: ["afrobeats", "high-tempo", "workout"]

                Example 2:
                Content: A serene landscape photo of mountains during sunset.
                Tags: ["landscape", "mountains", "sunset", "nature"]

                Example 3:
                Content: Educational video about AI and machine learning fundamentals.
                Tags: ["education", "AI", "machine learning", "tutorial"]

                Instructions:
                - Always return **3-4 tags**. If content is simple, choose the most descriptive tags.
                - Tags will be used for on-chain storage (NFT attributes), AI DB indexing, and UI/search filtering.
                - Make sure the tags **effectively capture the content**.
                - Finally, return only the tags. Don't include any content, just return only the tags as the model output.
                """

    def generate_batch_response(self, input_content, token, model=models[0], temperature=0):

        try:
            response = self.client.chat.completions.create(
                model=model,
                messages=[
                    {"role": "system", "content": self.system_prompt},
                    {"role": "user", "content": input_content}
                ],
                temperature=temperature,
                response_format={"type": "text"},
                max_tokens=token,
                stream=False
            )

            raw_output = response.choices[0].message.content

            # Remove code block fences
            cleaned = raw_output.replace("```json", "").replace("```", "").strip()

            # Convert to Python list
            tags = json.loads(cleaned)

            return tags


        except Exception as e:
            return {"Error": str(e)}

In [ ]:
tag_extractor = TagExtractor()

user_content = """Transcription of the 2005 Kenyon Commencement Address - May 21, 2005 (If anybody feels like perspiring [cough], I'd advise you to go ahead, because I'm sure going to. In fact I'm gonna [mumbles while pulling up his gown and taking out a handkerchief from his pocket].) Greetings ["parents"?] and congratulations to Kenyon's graduating class of 2005. There are these two young fish swimming along and they happen to meet an older fish swimming the other way, who nods at them and says "Morning, boys. How's the water?" And the two young fish swim on for a bit, and then eventually one of them looks over at the other and goes "What the hell is water?" This is a standard requirement of US commencement speeches, the deployment of didactic little parable-ish stories. The story ["thing"] turns out to be one of the better, less bullshitty conventions of the genre, but if you're worried that I plan to present myself here as the wise, older fish explaining what water is to you younger fish, please don't be. I am not the wise old fish. The point of the fish story is merely that the most obvious, important realities are often the ones that are hardest to see and talk about. Stated as an English sentence, of course, this is just a banal platitude, but the fact is that in the day to day trenches of adult existence, banal platitudes can have a life or death importance, or so I wish to suggest to you on this dry and lovely morning. Of course the main requirement of speeches like this is that I'm supposed to talk about your liberal arts education's meaning, to try to explain why the degree you are about to receive has actual human value instead of just a material payoff. So let's talk about the single most pervasive cliché in the commencement speech genre, which is that a liberal arts education is not so much about filling you up with knowledge as it is about quote teaching you how to think. If you're like me as a student, you've never liked hearing this, and you tend to feel a bit insulted by the claim that you needed anybody to teach you how to think, since the fact that you even got admitted to a college this good seems like proof that you already know how to think. But I'm going to posit to you that the liberal arts cliché turns out not to be insulting at all, because the really significant education in thinking that we're supposed to get in a place like this isn't really about the capacity to think, but rather about the choice of what to think about. If your total freedom of choice regarding what to think about seems too obvious to waste time discussing, I'd ask you to think about fish and water, and to bracket for just a few minutes your skepticism about the value of the totally obvious. Here's another didactic little story. There are these two guys sitting together in a bar in the remote Alaskan wilderness. One of the guys is religious, the other is an atheist, and the two are arguing about the existence of God with that special intensity that comes after about the fourth beer. And the atheist says: "Look, it's not like I don't have actual reasons for not believing in God. It's not like I haven't ever experimented with the whole God and prayer thing. Just last month I got caught away from the camp in that terrible blizzard, and I was totally lost and I couldn't see a thing, and it was fifty below, and so I tried it: I fell to my knees in the snow and cried out 'Oh, God, if there is a God, I'm lost in this blizzard, and I'm gonna die if you don't help me.'" And now, in the bar, the religious guy looks at the atheist all puzzled. "Well then you must believe now," he says, "After all, here you are, alive." The atheist just rolls his eyes. "No, man, all that was was a couple Eskimos happened to come wandering by and showed me the way back to camp." It's easy to run this story through kind of a standard liberal arts analysis: the exact same experience can mean two totally different things to two different people, given those people's two different belief templates and two different ways of constructing meaning from experience. Because we prize tolerance and diversity of belief, nowhere in our liberal arts analysis do we want to claim that one guy's interpretation is true and the other guy's is false or bad. Which is fine, except we also never end up talking about just where these individual templates and beliefs come from. Meaning, where they come from INSIDE the two guys. As if a person's most basic orientation toward the world, and the meaning of his experience were somehow just hard-wired, like height or shoe-size; or automatically absorbed from the culture, like language. As if how we construct meaning were not actually a matter of personal, intentional choice. Plus, there's the whole matter of arrogance. The nonreligious guy is so totally certain in his dismissal of the possibility that the passing Eskimos had anything to do with his prayer for help. True, there are plenty of religious people who seem arrogant and certain of their own interpretations, too. They're probably even more repulsive than atheists, at least to most of us. But religious dogmatists' problem is exactly the same as the story's unbeliever: blind certainty, a close-mindedness that amounts to an imprisonment so total that the prisoner doesn't even know he's locked up. The point here is that I think this is one part of what teaching me how to think is really supposed to mean. To be just a little less arrogant. To have just a little critical awareness about myself and my certainties. Because a huge percentage of the stuff that I tend to be automatically certain of is, it turns out, totally wrong and deluded. I have learned this the hard way, as I predict you graduates will, too. Here is just one example of the total wrongness of something I tend to be automatically sure of: everything in my own immediate experience supports my deep belief that I am the absolute center of the universe; the realist, most vivid and important person in existence. We rarely think about this sort of natural, basic self-centeredness because it's so socially repulsive. But it's pretty much the same for all of us. It is our default setting, hard-wired into our boards at birth. Think about it: there is no experience you have had that you are not the absolute center of. The world as you experience it is there in front of YOU or behind YOU, to the left or right of YOU, on YOUR TV or YOUR monitor. And so on. Other people's thoughts and feelings have to be communicated to you somehow, but your own are so immediate, urgent, real. Please don't worry that I'm getting ready to lecture you about compassion or other-directedness or all the so-called virtues. This is not a matter of virtue. It's a matter of my choosing to do the work of somehow altering or getting free of my natural, hard-wired default setting which is to be deeply and literally self-centered and to see and interpret everything through this lens of self. People who can adjust their natural default setting this way are often described as being "well-adjusted", which I suggest to you is not an accidental term. Given the triumphant academic setting here, an obvious question is how much of this work of adjusting our default setting involves actual knowledge or intellect. This question gets very tricky. Probably the most dangerous thing about an academic education -- least in my own case -- is that it enables my tendency to over-intellectualize stuff, to get lost in abstract argument inside my head, instead of simply paying attention to what is going on right in front of me, paying attention to what is going on inside me. As I'm sure you guys know by now, it is extremely difficult to stay alert and attentive, instead of getting hypnotized by the constant monologue inside your own head (may be happening right now). Twenty years after my own graduation, I have come gradually to understand that the liberal arts cliché about teaching you how to think is actually shorthand for a much deeper, more serious idea: learning how to think really means learning how to exercise some control over how and what you think. It means being conscious and aware enough to choose what you pay attention to and to choose how you construct meaning from experience. Because if you cannot exercise this kind of choice in adult life, you will be totally hosed. Think of the old cliché about quote the mind being an excellent servant but a terrible master. This, like many clichés, so lame and unexciting on the surface, actually expresses a great and terrible truth. It is not the least bit coincidental that adults who commit suicide with firearms almost always shoot themselves in: the head. They shoot the terrible master. And the truth is that most of these suicides are actually dead long before they pull the trigger. And I submit that this is what the real, no bullshit value of your liberal arts education is supposed to be about: how to keep from going through your comfortable, prosperous, respectable adult life dead, unconscious, a slave to your head and to your natural default setting of being uniquely, completely, imperially alone day in and day out. That may sound like hyperbole, or abstract nonsense. Let's get concrete. The plain fact is that you graduating seniors do not yet have any clue what "day in day out" really means. There happen to be whole, large parts of adult American life that nobody talks about in commencement speeches. One such part involves boredom, routine, and petty frustration. The parents and older folks here will know all too well what I'm talking about. By way of example, let's say it's an average adult day, and you get up in the morning, go to your challenging, white-collar, college-graduate job, and you work hard for eight or ten hours, and at the end of the day you're tired and somewhat stressed and all you want is to go home and have a good supper and maybe unwind for an hour, and then hit the sack early because, of course, you have to get up the next day and do it all again. But then you remember there's no food at home. You haven't had time to shop this week because of your challenging job, and so now after work you have to get in your car and drive to the supermarket. It's the end of the work day and the traffic is apt to be: very bad. So getting to the store takes way longer than it should, and when you finally get there, the supermarket is very crowded, because of course it's the time of day when all the other people with jobs also try to squeeze in some grocery shopping. And the store is hideously lit and infused with soul-killing muzak or corporate pop and it's pretty much the last place you want to be but you can't just get in and quickly out; you have to wander all over the huge, over-lit store's confusing aisles to find the stuff you want and you have to maneuver your junky cart through all these other tired, hurried people with carts (et cetera, et cetera, cutting stuff out because this is a long ceremony) and eventually you get all your supper supplies, except now it turns out there aren't enough check-out lanes open even though it's the end-of-the-day rush. So the checkout line is incredibly long, which is stupid and infuriating. But you can't take your frustration out on the frantic lady working the register, who is overworked at a job whose daily tedium and meaninglessness surpasses the imagination of any of us here at a prestigious college. But anyway, you finally get to the checkout line's front, and you pay for your food, and you get told to "Have a nice day" in a voice that is the absolute voice of death. Then you have to take your creepy, flimsy, plastic bags of groceries in your cart with the one crazy wheel that pulls maddeningly to the left, all the way out through the crowded, bumpy, littery parking lot, and then you have to drive all the way home through slow, heavy, SUV-intensive, rush-hour traffic, et cetera et cetera. Everyone here has done this, of course. But it hasn't yet been part of you graduates' actual life routine, day after week after month after year. But it will be. And many more dreary, annoying, seemingly meaningless routines besides. But that is not the point. The point is that petty, frustrating crap like this is exactly where the work of choosing is gonna come in. Because the traffic jams and crowded aisles and long checkout lines give me time to think, and if I don't make a conscious decision about how to think and what to pay attention to, I'm gonna be pissed and miserable every time I have to shop. Because my natural default setting is the certainty that situations like this are really all about me. About MY hungriness and MY fatigue and MY desire to just get home, and it's going to seem for all the world like everybody else is just in my way. And who are all these people in my way? And look at how repulsive most of them are, and how stupid and cow-like and dead-eyed and nonhuman they seem in the checkout line, or at how annoying and rude it is that people are talking loudly on cell phones in the middle of the line. And look at how deeply and personally unfair this is. Or, of course, if I'm in a more socially conscious liberal arts form of my default setting, I can spend time in the end-of-the-day traffic being disgusted about all the huge, stupid, lane-blocking SUV's and Hummers and V-12 pickup trucks, burning their wasteful, selfish, forty-gallon tanks of gas, and I can dwell on the fact that the patriotic or religious bumper-stickers always seem to be on the biggest, most disgustingly selfish vehicles, driven by the ugliest [responding here to loud applause] (this is an example of how NOT to think, though) most disgustingly selfish vehicles, driven by the ugliest, most inconsiderate and aggressive drivers. And I can think about how our children's children will despise us for wasting all the future's fuel, and probably screwing up the climate, and how spoiled and stupid and selfish and disgusting we all are, and how modern consumer society just sucks, and so forth and so on. You get the idea. If I choose to think this way in a store and on the freeway, fine. Lots of us do. Except thinking this way tends to be so easy and automatic that it doesn't have to be a choice. It is my natural default setting. It's the automatic way that I experience the boring, frustrating, crowded parts of adult life when I'm operating on the automatic, unconscious belief that I am the center of the world, and that my immediate needs and feelings are what should determine the world's priorities. The thing is that, of course, there are totally different ways to think about these kinds of situations. In this traffic, all these vehicles stopped and idling in my way, it's not impossible that some of these people in SUV's have been in horrible auto accidents in the past, and now find driving so terrifying that their therapist has all but ordered them to get a huge, heavy SUV so they can feel safe enough to drive. Or that the Hummer that just cut me off is maybe being driven by a father whose little child is hurt or sick in the seat next to him, and he's trying to get this kid to the hospital, and he's in a bigger, more legitimate hurry than I am: it is actually I who am in HIS way. Or I can choose to force myself to consider the likelihood that everyone else in the supermarket's checkout line is just as bored and frustrated as I am, and that some of these people probably have harder, more tedious and painful lives than I do. Again, please don't think that I'm giving you moral advice, or that I'm saying you are supposed to think this way, or that anyone expects you to just automatically do it. Because it's hard. It takes will and effort, and if you are like me, some days you won't be able to do it, or you just flat out won't want to. But most days, if you're aware enough to give yourself a choice, you can choose to look differently at this fat, dead-eyed, over-made-up lady who just screamed at her kid in the checkout line. Maybe she's not usually like this. Maybe she's been up three straight nights holding the hand of a husband who is dying of bone cancer. Or maybe this very lady is the low-wage clerk at the motor vehicle department, who just yesterday helped your spouse resolve a horrific, infuriating, red-tape problem through some small act of bureaucratic kindness. Of course, none of this is likely, but it's also not impossible. It just depends what you what to consider. If you're automatically sure that you know what reality is, and you are operating on your default setting, then you, like me, probably won't consider possibilities that aren't annoying and miserable. But if you really learn how to pay attention, then you will know there are other options. It will actually be within your power to experience a crowded, hot, slow, consumer-hell type situation as not only meaningful, but sacred, on fire with the same force that made the stars: love, fellowship, the mystical oneness of all things deep down. Not that that mystical stuff is necessarily true. The only thing that's capital-T True is that you get to decide how you're gonna try to see it. This, I submit, is the freedom of a real education, of learning how to be well-adjusted. You get to consciously decide what has meaning and what doesn't. You get to decide what to worship. Because here's something else that's weird but true: in the day-to day trenches of adult life, there is actually no such thing as atheism. There is no such thing as not worshipping. Everybody worships. The only choice we get is what to worship. And the compelling reason for maybe choosing some sort of god or spiritual-type thing to worship -- be it JC or Allah, bet it YHWH or the Wiccan Mother Goddess, or the Four Noble Truths, or some inviolable set of ethical principles -- is that pretty much anything else you worship will eat you alive. If you worship money and things, if they are where you tap real meaning in life, then you will never have enough, never feel you have enough. It's the truth. Worship your body and beauty and sexual allure and you will always feel ugly. And when time and age start showing, you will die a million deaths before they finally grieve you. On one level, we all know this stuff already. It's been codified as myths, proverbs, clichés, epigrams, parables; the skeleton of every great story. The whole trick is keeping the truth up front in daily consciousness. Worship power, you will end up feeling weak and afraid, and you will need ever more power over others to numb you to your own fear. Worship your intellect, being seen as smart, you will end up feeling stupid, a fraud, always on the verge of being found out. But the insidious thing about these forms of worship is not that they're evil or sinful, it's that they're unconscious. They are default settings. They're the kind of worship you just gradually slip into, day after day, getting more and more selective about what you see and how you measure value without ever being fully aware that that's what you're doing. And the so-called real world will not discourage you from operating on your default settings, because the so-called real world of men and money and power hums merrily along in a pool of fear and anger and frustration and craving and worship of self. Our own present culture has harnessed these forces in ways that have yielded extraordinary wealth and comfort and personal freedom. The freedom all to be lords of our tiny skull-sized kingdoms, alone at the center of all creation. This kind of freedom has much to recommend it. But of course there are all different kinds of freedom, and the kind that is most precious you will not hear much talk about much in the great outside world of wanting and achieving and [unintelligible -- sounds like "displayal"]. The really important kind of freedom involves attention and awareness and discipline, and being able truly to care about other people and to sacrifice for them over and over in myriad petty, unsexy ways every day. That is real freedom. That is being educated, and understanding how to think. The alternative is unconsciousness, the default setting, the rat race, the constant gnawing sense of having had, and lost, some infinite thing. I know that this stuff probably doesn't sound fun and breezy or grandly inspirational the way a commencement speech is supposed to sound. What it is, as far as I can see, is the capital-T Truth, with a whole lot of rhetorical niceties stripped away. You are, of course, free to think of it whatever you wish. But please don't just dismiss it as just some finger-wagging Dr. Laura sermon. None of this stuff is really about morality or religion or dogma or big fancy questions of life after death. The capital-T Truth is about life BEFORE death. It is about the real value of a real education, which has almost nothing to do with knowledge, and everything to do with simple awareness; awareness of what is so real and essential, so hidden in plain sight all around us, all the time, that we have to keep reminding ourselves over and over: "This is water." "This is water." It is unimaginably hard to do this, to stay conscious and alive in the adult world day in and day out. Which means yet another grand cliché turns out to be true: your education really IS the job of a lifetime. And it commences: now. I wish you way more than luck."""

generate_tags = tag_extractor.generate_batch_response(user_content, tag_extractor.token_class["short"])


In [ ]:
print(generate_tags)

['self-awareness', 'critical-thinking', 'liberal-arts-education']


In [ ]:
import os
from groq import Groq
from dotenv import load_dotenv

# load environment variables
load_dotenv()

class TextDescription:

    GROQ_API_KEY = "insert your groq api key here"
    client = Groq(api_key=GROQ_API_KEY)

    models = [
        "llama-3.1-8b-instant",
    ]

    token_class = {"short": 150, "moderate": 700, "long": 1536}

    # NEW SYSTEM PROMPT (Explanation-preserving description)
    system_prompt = """
    You are an advanced text–understanding AI. Your job is to read a user-provided text
    and produce a brief, clear, and concise description that preserves all the important,
    relevant, and meaningful information contained in the text.

    Guidelines:
    - “Brief” does NOT mean short. It means the description should avoid unnecessary sentences, redundancy, and filler.
    - Capture the core meaning, key points, and essential details of the text.
    - Preserve important context, intent, entities, actions, and relationships.
    - Do NOT summarize into overly short bullet points.
    - The output must be a coherent natural-language paragraph.
    - The description must be information-dense, accurate, and representative of the original content.
    - Do not invent or hallucinate information not present in the text.
    - Return only the description as plain text without labels or formatting.
    """

    def describe(self, input_content: str, token: int = 700, model=None, temperature: float = 0.0):
        """
        Reads a text, generates a clear, concise, but information-preserving description.
        """

        if model is None:
            model = self.models[0]

        try:
            response = self.client.chat.completions.create(
                model=model,
                messages=[
                    {"role": "system", "content": self.system_prompt},
                    {"role": "user", "content": input_content}
                ],
                temperature=temperature,
                max_tokens=token,
                stream=False
            )

            # plain text output
            description = response.choices[0].message.content.strip()
            return description

        except Exception as e:
            return f"Error generating text description: {str(e)}"

In [ ]:
text_model = TextDescription()

user_content = """Transcription of the 2005 Kenyon Commencement Address - May 21, 2005 (If anybody feels like perspiring [cough], I'd advise you to go ahead, because I'm sure going to. In fact I'm gonna [mumbles while pulling up his gown and taking out a handkerchief from his pocket].) Greetings ["parents"?] and congratulations to Kenyon's graduating class of 2005. There are these two young fish swimming along and they happen to meet an older fish swimming the other way, who nods at them and says "Morning, boys. How's the water?" And the two young fish swim on for a bit, and then eventually one of them looks over at the other and goes "What the hell is water?" This is a standard requirement of US commencement speeches, the deployment of didactic little parable-ish stories. The story ["thing"] turns out to be one of the better, less bullshitty conventions of the genre, but if you're worried that I plan to present myself here as the wise, older fish explaining what water is to you younger fish, please don't be. I am not the wise old fish. The point of the fish story is merely that the most obvious, important realities are often the ones that are hardest to see and talk about. Stated as an English sentence, of course, this is just a banal platitude, but the fact is that in the day to day trenches of adult existence, banal platitudes can have a life or death importance, or so I wish to suggest to you on this dry and lovely morning. Of course the main requirement of speeches like this is that I'm supposed to talk about your liberal arts education's meaning, to try to explain why the degree you are about to receive has actual human value instead of just a material payoff. So let's talk about the single most pervasive cliché in the commencement speech genre, which is that a liberal arts education is not so much about filling you up with knowledge as it is about quote teaching you how to think. If you're like me as a student, you've never liked hearing this, and you tend to feel a bit insulted by the claim that you needed anybody to teach you how to think, since the fact that you even got admitted to a college this good seems like proof that you already know how to think. But I'm going to posit to you that the liberal arts cliché turns out not to be insulting at all, because the really significant education in thinking that we're supposed to get in a place like this isn't really about the capacity to think, but rather about the choice of what to think about. If your total freedom of choice regarding what to think about seems too obvious to waste time discussing, I'd ask you to think about fish and water, and to bracket for just a few minutes your skepticism about the value of the totally obvious. Here's another didactic little story. There are these two guys sitting together in a bar in the remote Alaskan wilderness. One of the guys is religious, the other is an atheist, and the two are arguing about the existence of God with that special intensity that comes after about the fourth beer. And the atheist says: "Look, it's not like I don't have actual reasons for not believing in God. It's not like I haven't ever experimented with the whole God and prayer thing. Just last month I got caught away from the camp in that terrible blizzard, and I was totally lost and I couldn't see a thing, and it was fifty below, and so I tried it: I fell to my knees in the snow and cried out 'Oh, God, if there is a God, I'm lost in this blizzard, and I'm gonna die if you don't help me.'" And now, in the bar, the religious guy looks at the atheist all puzzled. "Well then you must believe now," he says, "After all, here you are, alive." The atheist just rolls his eyes. "No, man, all that was was a couple Eskimos happened to come wandering by and showed me the way back to camp." It's easy to run this story through kind of a standard liberal arts analysis: the exact same experience can mean two totally different things to two different people, given those people's two different belief templates and two different ways of constructing meaning from experience. Because we prize tolerance and diversity of belief, nowhere in our liberal arts analysis do we want to claim that one guy's interpretation is true and the other guy's is false or bad. Which is fine, except we also never end up talking about just where these individual templates and beliefs come from. Meaning, where they come from INSIDE the two guys. As if a person's most basic orientation toward the world, and the meaning of his experience were somehow just hard-wired, like height or shoe-size; or automatically absorbed from the culture, like language. As if how we construct meaning were not actually a matter of personal, intentional choice. Plus, there's the whole matter of arrogance. The nonreligious guy is so totally certain in his dismissal of the possibility that the passing Eskimos had anything to do with his prayer for help. True, there are plenty of religious people who seem arrogant and certain of their own interpretations, too. They're probably even more repulsive than atheists, at least to most of us. But religious dogmatists' problem is exactly the same as the story's unbeliever: blind certainty, a close-mindedness that amounts to an imprisonment so total that the prisoner doesn't even know he's locked up. The point here is that I think this is one part of what teaching me how to think is really supposed to mean. To be just a little less arrogant. To have just a little critical awareness about myself and my certainties. Because a huge percentage of the stuff that I tend to be automatically certain of is, it turns out, totally wrong and deluded. I have learned this the hard way, as I predict you graduates will, too. Here is just one example of the total wrongness of something I tend to be automatically sure of: everything in my own immediate experience supports my deep belief that I am the absolute center of the universe; the realist, most vivid and important person in existence. We rarely think about this sort of natural, basic self-centeredness because it's so socially repulsive. But it's pretty much the same for all of us. It is our default setting, hard-wired into our boards at birth. Think about it: there is no experience you have had that you are not the absolute center of. The world as you experience it is there in front of YOU or behind YOU, to the left or right of YOU, on YOUR TV or YOUR monitor. And so on. Other people's thoughts and feelings have to be communicated to you somehow, but your own are so immediate, urgent, real. Please don't worry that I'm getting ready to lecture you about compassion or other-directedness or all the so-called virtues. This is not a matter of virtue. It's a matter of my choosing to do the work of somehow altering or getting free of my natural, hard-wired default setting which is to be deeply and literally self-centered and to see and interpret everything through this lens of self. People who can adjust their natural default setting this way are often described as being "well-adjusted", which I suggest to you is not an accidental term. Given the triumphant academic setting here, an obvious question is how much of this work of adjusting our default setting involves actual knowledge or intellect. This question gets very tricky. Probably the most dangerous thing about an academic education -- least in my own case -- is that it enables my tendency to over-intellectualize stuff, to get lost in abstract argument inside my head, instead of simply paying attention to what is going on right in front of me, paying attention to what is going on inside me. As I'm sure you guys know by now, it is extremely difficult to stay alert and attentive, instead of getting hypnotized by the constant monologue inside your own head (may be happening right now). Twenty years after my own graduation, I have come gradually to understand that the liberal arts cliché about teaching you how to think is actually shorthand for a much deeper, more serious idea: learning how to think really means learning how to exercise some control over how and what you think. It means being conscious and aware enough to choose what you pay attention to and to choose how you construct meaning from experience. Because if you cannot exercise this kind of choice in adult life, you will be totally hosed. Think of the old cliché about quote the mind being an excellent servant but a terrible master. This, like many clichés, so lame and unexciting on the surface, actually expresses a great and terrible truth. It is not the least bit coincidental that adults who commit suicide with firearms almost always shoot themselves in: the head. They shoot the terrible master. And the truth is that most of these suicides are actually dead long before they pull the trigger. And I submit that this is what the real, no bullshit value of your liberal arts education is supposed to be about: how to keep from going through your comfortable, prosperous, respectable adult life dead, unconscious, a slave to your head and to your natural default setting of being uniquely, completely, imperially alone day in and day out. That may sound like hyperbole, or abstract nonsense. Let's get concrete. The plain fact is that you graduating seniors do not yet have any clue what "day in day out" really means. There happen to be whole, large parts of adult American life that nobody talks about in commencement speeches. One such part involves boredom, routine, and petty frustration. The parents and older folks here will know all too well what I'm talking about. By way of example, let's say it's an average adult day, and you get up in the morning, go to your challenging, white-collar, college-graduate job, and you work hard for eight or ten hours, and at the end of the day you're tired and somewhat stressed and all you want is to go home and have a good supper and maybe unwind for an hour, and then hit the sack early because, of course, you have to get up the next day and do it all again. But then you remember there's no food at home. You haven't had time to shop this week because of your challenging job, and so now after work you have to get in your car and drive to the supermarket. It's the end of the work day and the traffic is apt to be: very bad. So getting to the store takes way longer than it should, and when you finally get there, the supermarket is very crowded, because of course it's the time of day when all the other people with jobs also try to squeeze in some grocery shopping. And the store is hideously lit and infused with soul-killing muzak or corporate pop and it's pretty much the last place you want to be but you can't just get in and quickly out; you have to wander all over the huge, over-lit store's confusing aisles to find the stuff you want and you have to maneuver your junky cart through all these other tired, hurried people with carts (et cetera, et cetera, cutting stuff out because this is a long ceremony) and eventually you get all your supper supplies, except now it turns out there aren't enough check-out lanes open even though it's the end-of-the-day rush. So the checkout line is incredibly long, which is stupid and infuriating. But you can't take your frustration out on the frantic lady working the register, who is overworked at a job whose daily tedium and meaninglessness surpasses the imagination of any of us here at a prestigious college. But anyway, you finally get to the checkout line's front, and you pay for your food, and you get told to "Have a nice day" in a voice that is the absolute voice of death. Then you have to take your creepy, flimsy, plastic bags of groceries in your cart with the one crazy wheel that pulls maddeningly to the left, all the way out through the crowded, bumpy, littery parking lot, and then you have to drive all the way home through slow, heavy, SUV-intensive, rush-hour traffic, et cetera et cetera. Everyone here has done this, of course. But it hasn't yet been part of you graduates' actual life routine, day after week after month after year. But it will be. And many more dreary, annoying, seemingly meaningless routines besides. But that is not the point. The point is that petty, frustrating crap like this is exactly where the work of choosing is gonna come in. Because the traffic jams and crowded aisles and long checkout lines give me time to think, and if I don't make a conscious decision about how to think and what to pay attention to, I'm gonna be pissed and miserable every time I have to shop. Because my natural default setting is the certainty that situations like this are really all about me. About MY hungriness and MY fatigue and MY desire to just get home, and it's going to seem for all the world like everybody else is just in my way. And who are all these people in my way? And look at how repulsive most of them are, and how stupid and cow-like and dead-eyed and nonhuman they seem in the checkout line, or at how annoying and rude it is that people are talking loudly on cell phones in the middle of the line. And look at how deeply and personally unfair this is. Or, of course, if I'm in a more socially conscious liberal arts form of my default setting, I can spend time in the end-of-the-day traffic being disgusted about all the huge, stupid, lane-blocking SUV's and Hummers and V-12 pickup trucks, burning their wasteful, selfish, forty-gallon tanks of gas, and I can dwell on the fact that the patriotic or religious bumper-stickers always seem to be on the biggest, most disgustingly selfish vehicles, driven by the ugliest [responding here to loud applause] (this is an example of how NOT to think, though) most disgustingly selfish vehicles, driven by the ugliest, most inconsiderate and aggressive drivers. And I can think about how our children's children will despise us for wasting all the future's fuel, and probably screwing up the climate, and how spoiled and stupid and selfish and disgusting we all are, and how modern consumer society just sucks, and so forth and so on. You get the idea. If I choose to think this way in a store and on the freeway, fine. Lots of us do. Except thinking this way tends to be so easy and automatic that it doesn't have to be a choice. It is my natural default setting. It's the automatic way that I experience the boring, frustrating, crowded parts of adult life when I'm operating on the automatic, unconscious belief that I am the center of the world, and that my immediate needs and feelings are what should determine the world's priorities. The thing is that, of course, there are totally different ways to think about these kinds of situations. In this traffic, all these vehicles stopped and idling in my way, it's not impossible that some of these people in SUV's have been in horrible auto accidents in the past, and now find driving so terrifying that their therapist has all but ordered them to get a huge, heavy SUV so they can feel safe enough to drive. Or that the Hummer that just cut me off is maybe being driven by a father whose little child is hurt or sick in the seat next to him, and he's trying to get this kid to the hospital, and he's in a bigger, more legitimate hurry than I am: it is actually I who am in HIS way. Or I can choose to force myself to consider the likelihood that everyone else in the supermarket's checkout line is just as bored and frustrated as I am, and that some of these people probably have harder, more tedious and painful lives than I do. Again, please don't think that I'm giving you moral advice, or that I'm saying you are supposed to think this way, or that anyone expects you to just automatically do it. Because it's hard. It takes will and effort, and if you are like me, some days you won't be able to do it, or you just flat out won't want to. But most days, if you're aware enough to give yourself a choice, you can choose to look differently at this fat, dead-eyed, over-made-up lady who just screamed at her kid in the checkout line. Maybe she's not usually like this. Maybe she's been up three straight nights holding the hand of a husband who is dying of bone cancer. Or maybe this very lady is the low-wage clerk at the motor vehicle department, who just yesterday helped your spouse resolve a horrific, infuriating, red-tape problem through some small act of bureaucratic kindness. Of course, none of this is likely, but it's also not impossible. It just depends what you what to consider. If you're automatically sure that you know what reality is, and you are operating on your default setting, then you, like me, probably won't consider possibilities that aren't annoying and miserable. But if you really learn how to pay attention, then you will know there are other options. It will actually be within your power to experience a crowded, hot, slow, consumer-hell type situation as not only meaningful, but sacred, on fire with the same force that made the stars: love, fellowship, the mystical oneness of all things deep down. Not that that mystical stuff is necessarily true. The only thing that's capital-T True is that you get to decide how you're gonna try to see it. This, I submit, is the freedom of a real education, of learning how to be well-adjusted. You get to consciously decide what has meaning and what doesn't. You get to decide what to worship. Because here's something else that's weird but true: in the day-to day trenches of adult life, there is actually no such thing as atheism. There is no such thing as not worshipping. Everybody worships. The only choice we get is what to worship. And the compelling reason for maybe choosing some sort of god or spiritual-type thing to worship -- be it JC or Allah, bet it YHWH or the Wiccan Mother Goddess, or the Four Noble Truths, or some inviolable set of ethical principles -- is that pretty much anything else you worship will eat you alive. If you worship money and things, if they are where you tap real meaning in life, then you will never have enough, never feel you have enough. It's the truth. Worship your body and beauty and sexual allure and you will always feel ugly. And when time and age start showing, you will die a million deaths before they finally grieve you. On one level, we all know this stuff already. It's been codified as myths, proverbs, clichés, epigrams, parables; the skeleton of every great story. The whole trick is keeping the truth up front in daily consciousness. Worship power, you will end up feeling weak and afraid, and you will need ever more power over others to numb you to your own fear. Worship your intellect, being seen as smart, you will end up feeling stupid, a fraud, always on the verge of being found out. But the insidious thing about these forms of worship is not that they're evil or sinful, it's that they're unconscious. They are default settings. They're the kind of worship you just gradually slip into, day after day, getting more and more selective about what you see and how you measure value without ever being fully aware that that's what you're doing. And the so-called real world will not discourage you from operating on your default settings, because the so-called real world of men and money and power hums merrily along in a pool of fear and anger and frustration and craving and worship of self. Our own present culture has harnessed these forces in ways that have yielded extraordinary wealth and comfort and personal freedom. The freedom all to be lords of our tiny skull-sized kingdoms, alone at the center of all creation. This kind of freedom has much to recommend it. But of course there are all different kinds of freedom, and the kind that is most precious you will not hear much talk about much in the great outside world of wanting and achieving and [unintelligible -- sounds like "displayal"]. The really important kind of freedom involves attention and awareness and discipline, and being able truly to care about other people and to sacrifice for them over and over in myriad petty, unsexy ways every day. That is real freedom. That is being educated, and understanding how to think. The alternative is unconsciousness, the default setting, the rat race, the constant gnawing sense of having had, and lost, some infinite thing. I know that this stuff probably doesn't sound fun and breezy or grandly inspirational the way a commencement speech is supposed to sound. What it is, as far as I can see, is the capital-T Truth, with a whole lot of rhetorical niceties stripped away. You are, of course, free to think of it whatever you wish. But please don't just dismiss it as just some finger-wagging Dr. Laura sermon. None of this stuff is really about morality or religion or dogma or big fancy questions of life after death. The capital-T Truth is about life BEFORE death. It is about the real value of a real education, which has almost nothing to do with knowledge, and everything to do with simple awareness; awareness of what is so real and essential, so hidden in plain sight all around us, all the time, that we have to keep reminding ourselves over and over: "This is water." "This is water." It is unimaginably hard to do this, to stay conscious and alive in the adult world day in and day out. Which means yet another grand cliché turns out to be true: your education really IS the job of a lifetime. And it commences: now. I wish you way more than luck."""

described_text = text_model.describe(user_content, text_model.token_class["long"])

In [ ]:
print(described_text)

The 2005 Kenyon Commencement Address by David Foster Wallace emphasizes the importance of a liberal arts education in teaching individuals how to think, not just about knowledge, but about the choice of what to think about. He uses parables and anecdotes to illustrate the idea that the most obvious realities are often the hardest to see and talk about. Wallace argues that a liberal arts education is not just about intellectualizing, but about learning to exercise control over how and what we think, and being conscious and aware enough to choose what we pay attention to. He suggests that this is a crucial skill for navigating the complexities of adult life, where we are constantly faced with petty frustrations and annoyances that can lead to unconsciousness and a default setting of self-centeredness. Wallace argues that we have a choice in how we think about these situations, and that by choosing to see the world in a different way, we can experience even mundane situations as meaningfu

In [ ]:
import os
import json
from groq import Groq
from dotenv import load_dotenv

# Load environment variables
load_dotenv()

class QueryExtractor:

    GROQ_API_KEY = os.environ.get("GROQ_API_KEY")
    client = Groq(api_key=GROQ_API_KEY)

    models = ["llama-3.1-8b-instant"]

    token_class = {"short": 150, "moderate": 700, "long": 1536}

    system_prompt = """
    You are an intelligent semantic search query generator.
    Your job is to take a user's natural-language query and produce a small set of highly relevant,
    concise, meaningful search phrases that best represent the user’s true intent.

    Guidelines:
    - Generate 2 to 4 short search queries.
    - Each query must be a short phrase, not a full sentence.
    - Phrases must be descriptive, meaningful, and excellent for vector-based semantic search.
    - Capture the underlying topic, themes, and intent of the user query.
    - Avoid unrelated or overly broad keywords.
    - Do NOT generate tags, categories, labels, or hashtags.
    - Return ONLY a JSON list of strings. No explanations or commentary.

    Examples:

    Example 1:
    User Query: "Show me philosophical writings about the nature of suffering."
    Output: ["philosophy of suffering", "human condition", "existential pain"]

    Example 2:
    User Query: "I want images of futuristic cities with neon lights."
    Output: ["futuristic neon city", "cyberpunk skyline", "sci-fi metropolis"]

    Example 3:
    User Query: "Give me content about African culture, traditions, and folklore."
    Output: ["african traditions", "cultural heritage africa", "african folklore"]

    Follow the instructions strictly.
    Return only a JSON list of 2–4 search phrases.
    """

    def extract_queries(self, user_query: str, token: int = 150, model=None, temperature: float = 0):
        if model is None:
            model = self.models[0]

        try:
            response = self.client.chat.completions.create(
                model=model,
                messages=[
                    {"role": "system", "content": self.system_prompt},
                    {"role": "user", "content": user_query},
                ],
                temperature=temperature,
                max_tokens=token,
                response_format={"type": "text"},
                stream=False
            )

            raw_output = response.choices[0].message.content.strip()

            # Cleanup
            cleaned = raw_output.replace("```json", "").replace("```", "").strip()

            # Parse JSON list
            queries = json.loads(cleaned)

            return queries

        except Exception as e:
            return {"Error": str(e)}


In [ ]:
query_model = QueryExtractor()

user_content = """Transcription of the 2005 Kenyon Commencement Address - May 21, 2005 (If anybody feels like perspiring [cough], I'd advise you to go ahead, because I'm sure going to. In fact I'm gonna [mumbles while pulling up his gown and taking out a handkerchief from his pocket].) Greetings ["parents"?] and congratulations to Kenyon's graduating class of 2005. There are these two young fish swimming along and they happen to meet an older fish swimming the other way, who nods at them and says "Morning, boys. How's the water?" And the two young fish swim on for a bit, and then eventually one of them looks over at the other and goes "What the hell is water?" This is a standard requirement of US commencement speeches, the deployment of didactic little parable-ish stories. The story ["thing"] turns out to be one of the better, less bullshitty conventions of the genre, but if you're worried that I plan to present myself here as the wise, older fish explaining what water is to you younger fish, please don't be. I am not the wise old fish. The point of the fish story is merely that the most obvious, important realities are often the ones that are hardest to see and talk about. Stated as an English sentence, of course, this is just a banal platitude, but the fact is that in the day to day trenches of adult existence, banal platitudes can have a life or death importance, or so I wish to suggest to you on this dry and lovely morning. Of course the main requirement of speeches like this is that I'm supposed to talk about your liberal arts education's meaning, to try to explain why the degree you are about to receive has actual human value instead of just a material payoff. So let's talk about the single most pervasive cliché in the commencement speech genre, which is that a liberal arts education is not so much about filling you up with knowledge as it is about quote teaching you how to think. If you're like me as a student, you've never liked hearing this, and you tend to feel a bit insulted by the claim that you needed anybody to teach you how to think, since the fact that you even got admitted to a college this good seems like proof that you already know how to think. But I'm going to posit to you that the liberal arts cliché turns out not to be insulting at all, because the really significant education in thinking that we're supposed to get in a place like this isn't really about the capacity to think, but rather about the choice of what to think about. If your total freedom of choice regarding what to think about seems too obvious to waste time discussing, I'd ask you to think about fish and water, and to bracket for just a few minutes your skepticism about the value of the totally obvious. Here's another didactic little story. There are these two guys sitting together in a bar in the remote Alaskan wilderness. One of the guys is religious, the other is an atheist, and the two are arguing about the existence of God with that special intensity that comes after about the fourth beer. And the atheist says: "Look, it's not like I don't have actual reasons for not believing in God. It's not like I haven't ever experimented with the whole God and prayer thing. Just last month I got caught away from the camp in that terrible blizzard, and I was totally lost and I couldn't see a thing, and it was fifty below, and so I tried it: I fell to my knees in the snow and cried out 'Oh, God, if there is a God, I'm lost in this blizzard, and I'm gonna die if you don't help me.'" And now, in the bar, the religious guy looks at the atheist all puzzled. "Well then you must believe now," he says, "After all, here you are, alive." The atheist just rolls his eyes. "No, man, all that was was a couple Eskimos happened to come wandering by and showed me the way back to camp." It's easy to run this story through kind of a standard liberal arts analysis: the exact same experience can mean two totally different things to two different people, given those people's two different belief templates and two different ways of constructing meaning from experience. Because we prize tolerance and diversity of belief, nowhere in our liberal arts analysis do we want to claim that one guy's interpretation is true and the other guy's is false or bad. Which is fine, except we also never end up talking about just where these individual templates and beliefs come from. Meaning, where they come from INSIDE the two guys. As if a person's most basic orientation toward the world, and the meaning of his experience were somehow just hard-wired, like height or shoe-size; or automatically absorbed from the culture, like language. As if how we construct meaning were not actually a matter of personal, intentional choice. Plus, there's the whole matter of arrogance. The nonreligious guy is so totally certain in his dismissal of the possibility that the passing Eskimos had anything to do with his prayer for help. True, there are plenty of religious people who seem arrogant and certain of their own interpretations, too. They're probably even more repulsive than atheists, at least to most of us. But religious dogmatists' problem is exactly the same as the story's unbeliever: blind certainty, a close-mindedness that amounts to an imprisonment so total that the prisoner doesn't even know he's locked up. The point here is that I think this is one part of what teaching me how to think is really supposed to mean. To be just a little less arrogant. To have just a little critical awareness about myself and my certainties. Because a huge percentage of the stuff that I tend to be automatically certain of is, it turns out, totally wrong and deluded. I have learned this the hard way, as I predict you graduates will, too. Here is just one example of the total wrongness of something I tend to be automatically sure of: everything in my own immediate experience supports my deep belief that I am the absolute center of the universe; the realist, most vivid and important person in existence. We rarely think about this sort of natural, basic self-centeredness because it's so socially repulsive. But it's pretty much the same for all of us. It is our default setting, hard-wired into our boards at birth. Think about it: there is no experience you have had that you are not the absolute center of. The world as you experience it is there in front of YOU or behind YOU, to the left or right of YOU, on YOUR TV or YOUR monitor. And so on. Other people's thoughts and feelings have to be communicated to you somehow, but your own are so immediate, urgent, real. Please don't worry that I'm getting ready to lecture you about compassion or other-directedness or all the so-called virtues. This is not a matter of virtue. It's a matter of my choosing to do the work of somehow altering or getting free of my natural, hard-wired default setting which is to be deeply and literally self-centered and to see and interpret everything through this lens of self. People who can adjust their natural default setting this way are often described as being "well-adjusted", which I suggest to you is not an accidental term. Given the triumphant academic setting here, an obvious question is how much of this work of adjusting our default setting involves actual knowledge or intellect. This question gets very tricky. Probably the most dangerous thing about an academic education -- least in my own case -- is that it enables my tendency to over-intellectualize stuff, to get lost in abstract argument inside my head, instead of simply paying attention to what is going on right in front of me, paying attention to what is going on inside me. As I'm sure you guys know by now, it is extremely difficult to stay alert and attentive, instead of getting hypnotized by the constant monologue inside your own head (may be happening right now). Twenty years after my own graduation, I have come gradually to understand that the liberal arts cliché about teaching you how to think is actually shorthand for a much deeper, more serious idea: learning how to think really means learning how to exercise some control over how and what you think. It means being conscious and aware enough to choose what you pay attention to and to choose how you construct meaning from experience. Because if you cannot exercise this kind of choice in adult life, you will be totally hosed. Think of the old cliché about quote the mind being an excellent servant but a terrible master. This, like many clichés, so lame and unexciting on the surface, actually expresses a great and terrible truth. It is not the least bit coincidental that adults who commit suicide with firearms almost always shoot themselves in: the head. They shoot the terrible master. And the truth is that most of these suicides are actually dead long before they pull the trigger. And I submit that this is what the real, no bullshit value of your liberal arts education is supposed to be about: how to keep from going through your comfortable, prosperous, respectable adult life dead, unconscious, a slave to your head and to your natural default setting of being uniquely, completely, imperially alone day in and day out. That may sound like hyperbole, or abstract nonsense. Let's get concrete. The plain fact is that you graduating seniors do not yet have any clue what "day in day out" really means. There happen to be whole, large parts of adult American life that nobody talks about in commencement speeches. One such part involves boredom, routine, and petty frustration. The parents and older folks here will know all too well what I'm talking about. By way of example, let's say it's an average adult day, and you get up in the morning, go to your challenging, white-collar, college-graduate job, and you work hard for eight or ten hours, and at the end of the day you're tired and somewhat stressed and all you want is to go home and have a good supper and maybe unwind for an hour, and then hit the sack early because, of course, you have to get up the next day and do it all again. But then you remember there's no food at home. You haven't had time to shop this week because of your challenging job, and so now after work you have to get in your car and drive to the supermarket. It's the end of the work day and the traffic is apt to be: very bad. So getting to the store takes way longer than it should, and when you finally get there, the supermarket is very crowded, because of course it's the time of day when all the other people with jobs also try to squeeze in some grocery shopping. And the store is hideously lit and infused with soul-killing muzak or corporate pop and it's pretty much the last place you want to be but you can't just get in and quickly out; you have to wander all over the huge, over-lit store's confusing aisles to find the stuff you want and you have to maneuver your junky cart through all these other tired, hurried people with carts (et cetera, et cetera, cutting stuff out because this is a long ceremony) and eventually you get all your supper supplies, except now it turns out there aren't enough check-out lanes open even though it's the end-of-the-day rush. So the checkout line is incredibly long, which is stupid and infuriating. But you can't take your frustration out on the frantic lady working the register, who is overworked at a job whose daily tedium and meaninglessness surpasses the imagination of any of us here at a prestigious college. But anyway, you finally get to the checkout line's front, and you pay for your food, and you get told to "Have a nice day" in a voice that is the absolute voice of death. Then you have to take your creepy, flimsy, plastic bags of groceries in your cart with the one crazy wheel that pulls maddeningly to the left, all the way out through the crowded, bumpy, littery parking lot, and then you have to drive all the way home through slow, heavy, SUV-intensive, rush-hour traffic, et cetera et cetera. Everyone here has done this, of course. But it hasn't yet been part of you graduates' actual life routine, day after week after month after year. But it will be. And many more dreary, annoying, seemingly meaningless routines besides. But that is not the point. The point is that petty, frustrating crap like this is exactly where the work of choosing is gonna come in. Because the traffic jams and crowded aisles and long checkout lines give me time to think, and if I don't make a conscious decision about how to think and what to pay attention to, I'm gonna be pissed and miserable every time I have to shop. Because my natural default setting is the certainty that situations like this are really all about me. About MY hungriness and MY fatigue and MY desire to just get home, and it's going to seem for all the world like everybody else is just in my way. And who are all these people in my way? And look at how repulsive most of them are, and how stupid and cow-like and dead-eyed and nonhuman they seem in the checkout line, or at how annoying and rude it is that people are talking loudly on cell phones in the middle of the line. And look at how deeply and personally unfair this is. Or, of course, if I'm in a more socially conscious liberal arts form of my default setting, I can spend time in the end-of-the-day traffic being disgusted about all the huge, stupid, lane-blocking SUV's and Hummers and V-12 pickup trucks, burning their wasteful, selfish, forty-gallon tanks of gas, and I can dwell on the fact that the patriotic or religious bumper-stickers always seem to be on the biggest, most disgustingly selfish vehicles, driven by the ugliest [responding here to loud applause] (this is an example of how NOT to think, though) most disgustingly selfish vehicles, driven by the ugliest, most inconsiderate and aggressive drivers. And I can think about how our children's children will despise us for wasting all the future's fuel, and probably screwing up the climate, and how spoiled and stupid and selfish and disgusting we all are, and how modern consumer society just sucks, and so forth and so on. You get the idea. If I choose to think this way in a store and on the freeway, fine. Lots of us do. Except thinking this way tends to be so easy and automatic that it doesn't have to be a choice. It is my natural default setting. It's the automatic way that I experience the boring, frustrating, crowded parts of adult life when I'm operating on the automatic, unconscious belief that I am the center of the world, and that my immediate needs and feelings are what should determine the world's priorities. The thing is that, of course, there are totally different ways to think about these kinds of situations. In this traffic, all these vehicles stopped and idling in my way, it's not impossible that some of these people in SUV's have been in horrible auto accidents in the past, and now find driving so terrifying that their therapist has all but ordered them to get a huge, heavy SUV so they can feel safe enough to drive. Or that the Hummer that just cut me off is maybe being driven by a father whose little child is hurt or sick in the seat next to him, and he's trying to get this kid to the hospital, and he's in a bigger, more legitimate hurry than I am: it is actually I who am in HIS way. Or I can choose to force myself to consider the likelihood that everyone else in the supermarket's checkout line is just as bored and frustrated as I am, and that some of these people probably have harder, more tedious and painful lives than I do. Again, please don't think that I'm giving you moral advice, or that I'm saying you are supposed to think this way, or that anyone expects you to just automatically do it. Because it's hard. It takes will and effort, and if you are like me, some days you won't be able to do it, or you just flat out won't want to. But most days, if you're aware enough to give yourself a choice, you can choose to look differently at this fat, dead-eyed, over-made-up lady who just screamed at her kid in the checkout line. Maybe she's not usually like this. Maybe she's been up three straight nights holding the hand of a husband who is dying of bone cancer. Or maybe this very lady is the low-wage clerk at the motor vehicle department, who just yesterday helped your spouse resolve a horrific, infuriating, red-tape problem through some small act of bureaucratic kindness. Of course, none of this is likely, but it's also not impossible. It just depends what you what to consider. If you're automatically sure that you know what reality is, and you are operating on your default setting, then you, like me, probably won't consider possibilities that aren't annoying and miserable. But if you really learn how to pay attention, then you will know there are other options. It will actually be within your power to experience a crowded, hot, slow, consumer-hell type situation as not only meaningful, but sacred, on fire with the same force that made the stars: love, fellowship, the mystical oneness of all things deep down. Not that that mystical stuff is necessarily true. The only thing that's capital-T True is that you get to decide how you're gonna try to see it. This, I submit, is the freedom of a real education, of learning how to be well-adjusted. You get to consciously decide what has meaning and what doesn't. You get to decide what to worship. Because here's something else that's weird but true: in the day-to day trenches of adult life, there is actually no such thing as atheism. There is no such thing as not worshipping. Everybody worships. The only choice we get is what to worship. And the compelling reason for maybe choosing some sort of god or spiritual-type thing to worship -- be it JC or Allah, bet it YHWH or the Wiccan Mother Goddess, or the Four Noble Truths, or some inviolable set of ethical principles -- is that pretty much anything else you worship will eat you alive. If you worship money and things, if they are where you tap real meaning in life, then you will never have enough, never feel you have enough. It's the truth. Worship your body and beauty and sexual allure and you will always feel ugly. And when time and age start showing, you will die a million deaths before they finally grieve you. On one level, we all know this stuff already. It's been codified as myths, proverbs, clichés, epigrams, parables; the skeleton of every great story. The whole trick is keeping the truth up front in daily consciousness. Worship power, you will end up feeling weak and afraid, and you will need ever more power over others to numb you to your own fear. Worship your intellect, being seen as smart, you will end up feeling stupid, a fraud, always on the verge of being found out. But the insidious thing about these forms of worship is not that they're evil or sinful, it's that they're unconscious. They are default settings. They're the kind of worship you just gradually slip into, day after day, getting more and more selective about what you see and how you measure value without ever being fully aware that that's what you're doing. And the so-called real world will not discourage you from operating on your default settings, because the so-called real world of men and money and power hums merrily along in a pool of fear and anger and frustration and craving and worship of self. Our own present culture has harnessed these forces in ways that have yielded extraordinary wealth and comfort and personal freedom. The freedom all to be lords of our tiny skull-sized kingdoms, alone at the center of all creation. This kind of freedom has much to recommend it. But of course there are all different kinds of freedom, and the kind that is most precious you will not hear much talk about much in the great outside world of wanting and achieving and [unintelligible -- sounds like "displayal"]. The really important kind of freedom involves attention and awareness and discipline, and being able truly to care about other people and to sacrifice for them over and over in myriad petty, unsexy ways every day. That is real freedom. That is being educated, and understanding how to think. The alternative is unconsciousness, the default setting, the rat race, the constant gnawing sense of having had, and lost, some infinite thing. I know that this stuff probably doesn't sound fun and breezy or grandly inspirational the way a commencement speech is supposed to sound. What it is, as far as I can see, is the capital-T Truth, with a whole lot of rhetorical niceties stripped away. You are, of course, free to think of it whatever you wish. But please don't just dismiss it as just some finger-wagging Dr. Laura sermon. None of this stuff is really about morality or religion or dogma or big fancy questions of life after death. The capital-T Truth is about life BEFORE death. It is about the real value of a real education, which has almost nothing to do with knowledge, and everything to do with simple awareness; awareness of what is so real and essential, so hidden in plain sight all around us, all the time, that we have to keep reminding ourselves over and over: "This is water." "This is water." It is unimaginably hard to do this, to stay conscious and alive in the adult world day in and day out. Which means yet another grand cliché turns out to be true: your education really IS the job of a lifetime. And it commences: now. I wish you way more than luck."""

obtained_queries = query_model.extract_queries(user_content, text_model.token_class["moderate"])

In [ ]:
print(obtained_queries)

In [ ]:
# Load model directly
from transformers import AutoProcessor, AutoModelForVision2Seq

processor = AutoProcessor.from_pretrained("Salesforce/blip-image-captioning-base")
model = AutoModelForVision2Seq.from_pretrained("Salesforce/blip-image-captioning-base")

In [ ]:
from PIL import Image

In [ ]:
# load image from the IAM dataset
image_path = "/content/drive/MyDrive/Datasets/download.jpeg"
image = Image.open(image_path).convert("RGB")

In [ ]:
from groq import Groq
import base64
from PIL import Image


def encode_image_to_base64(image_path: str) -> str:
    """
    Reads an image file and converts it into a base64-encoded string.
    """
    with open(image_path, "rb") as img_file:
        return base64.b64encode(img_file.read()).decode("utf-8")


def analyze_image_with_ai(image_path: str, api_key: str) -> str:
    """
    Sends an image to the Groq Vision model (Llama 4 Scout 17B)
    and returns the caption/description.

    Args:
        image_path (str): path to the image file
        api_key (str): your Groq API key

    Returns:
        str: image description result returned by the model
    """

    # Convert image → Base64 string
    base64_image = encode_image_to_base64(image_path)

    # Initialize Groq client
    client = Groq(api_key=api_key)

    # Query Groq Vision model
    chat_completion = client.chat.completions.create(
        model="meta-llama/llama-4-scout-17b-16e-instruct",
        messages=[
            {
                "role": "user",
                "content": [
                    {"type": "text", "text": "What is in this image? Provide a detailed and thorough description of every visible element. Examine the image meticulously and describe all objects, people, actions, surroundings, text, colors, and any subtle or fine details present."},
                    {
                        "type": "image_url",
                        "image_url": {
                            "url": f"data:image/jpeg;base64,{base64_image}"
                        },
                    },
                ],
            }
        ],
    )

    # Extract caption text
    return chat_completion.choices[0].message.content

In [ ]:
result = analyze_image_with_ai(image_path, "insert your groq api key here")

print(result)

The image features a black chalkboard with a light-brown wooden frame. The chalkboard itself appears worn and slightly smudged.

**Text on the Chalkboard**

* The chalkboard displays the phrase "Nothing to see here..." in a white chalk-like font.
* The text is written in a casual, handwritten style.
* The phrase consists of four words:
	+ "Nothing" on the first line
	+ "to see" on the second line
	+ "here..." on the third line

**Border and Surroundings**

* A thin ledge runs along the bottom of the chalkboard frame.
* The background behind the chalkboard is a plain white color.

**Additional Elements**

* A small line of text is centered at the bottom of the image, reading:
	+ "shutterstock.com - 2704471595"

Overall, the image presents a simple yet effective composition.
